# TED Talks: Topic classification and Evolution tracking
|- final_v2.ipynb  
|- TEDLIUM_r1  
|-- README  
|-- stm  
|--- AaronHuey_2010X.stm  
|--- AdamGrosser_2007.stm  
|--- ...  

In [2]:
# Import libraries
import os
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from keybert import KeyBERT
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Load stm data

def extract_year(filename):
    match = re.search(r'_(\d{4})', filename)
    return int(match.group(1)) if match else None

def load_stm_to_df(stm_dir):
    data = []
    for file in sorted(os.listdir(stm_dir)):
        if file.endswith('.stm'):
            path = os.path.join(stm_dir, file)
            with open(path, 'r', encoding='utf-8') as f:
                lines = f.readlines()
                content = ' '.join([re.sub(r'\s+', ' ', l.strip().split(' ', 6)[-1]) for l in lines])
                year = extract_year(file)
                data.append({
                    'file_name': file,
                    'year': year,
                    'text': content
                })
    return pd.DataFrame(data)

df = load_stm_to_df("TEDLIUM_r1/stm")

In [4]:
df.to_csv("stm_text.csv", index=False, encoding='utf-8')

In [5]:
df

,file_name,year,text
0,AJJacobs_2007P.stm,2007,i've also had some meals that(2) make me want(...
1,AaronHuey_2010X.stm,2010,we have indeed {NOISE} taken <sil> the best pa...
2,AdamGrosser_2007.stm,2007,{NOISE} and then as(2) the(2) ammonia re(2) ev...
3,AdamSadowsky_2010X.stm,2010,<sil> we came up with(2) {UH} a list {COUGH} o...
4,AdamSavage_2008P.stm,2008,and(2) i thought to myself <sil> wouldn't it b...
...,...,...,...
769,YvesBehar_2009.stm,2009,{NOISE} the(2) beginning of any <sil> collabor...
770,ZainabSalbi_2010G.stm,2010,<sil> {SMACK} and {COUGH} killed {COUGH} him <...
771,ZeFrank_2004.stm,2004,and {UH} so the idea is {NOISE} that {UH} you ...
772,ZeFrank_2010G.stm,2010,{SMACK} this was(2) a mother 's(3) day <sil> (...


In [6]:
# Data Preprocessing

import re
import contractions
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer

# Initialize lemmatizer and stopword list
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english")) | {
    "so", "well", "okay", "right", "actually", "just", "like",
    "sort", "look", "ca", "nt", "cant", "dont", "do", "don"
}

# Non-verbal filler and noise tokens commonly found in transcripts
non_content_tokens = {
    'sil', 'noise', 'breathe', 'uh', 'um', 'ah', 'mmm', 'hmm', 'laugh', 'cough'
}

# ---------- Module 1: Clean structural and noisy artifacts ----------
def clean_text(text: str) -> str:
    """
    Removes HTML tags, speaker labels, timestamps, and other non-content symbols.
    """
    text = re.sub(r"<[^>]+>", "", text)                          # Remove HTML tags
    text = re.sub(r"\{[^}]+\}", "", text)                        # Remove content in {...}
    text = re.sub(r"\(\d+\)", "", text)                          # Remove numbered tags like (123)
    text = re.sub(r"\([^)]*?-\d+\.\d+-\d+\.\d+-[^\)]*\)", "", text)  # Remove complex timestamp-like patterns
    text = re.sub(r"\b[fF]\d+_[mMfF]\b", "", text)               # Remove speaker codes like F1_M
    text = re.sub(r"\bs\d+\b", "", text)                         # Remove speaker IDs like s1, s2
    text = re.sub(r"\s+", " ", text).strip()                     # Normalize whitespace
    return text

# ---------- Module 2: Full preprocessing pipeline ----------
def preprocess(text: str) -> str:
    """
    Full pipeline:
    1. Expand contractions
    2. Clean structure/noise
    3. Lowercase
    4. Sentence + word tokenization
    5. Remove stopwords and non-verbal tokens
    6. Lemmatize words
    """
    text = contractions.fix(text)           # Step 1: Expand contractions (e.g., "don't" → "do not")
    text = clean_text(text)                 # Step 2: Remove structural artifacts
    text = text.lower()                     # Step 3: Lowercase

    tokens_out = []
    for sent in sent_tokenize(text):        # Step 4: Sentence tokenization
        for w in word_tokenize(sent):       # Step 4: Word tokenization
            w = re.sub(r"[^a-z]", "", w)    # Remove non-alphabetic characters
            if not w or w in stop_words or w in non_content_tokens:
                continue                    # Step 5: Filter out stopwords and fillers
            lemma = lemmatizer.lemmatize(w) # Step 6: Lemmatize
            tokens_out.append(lemma)

    return " ".join(tokens_out)

# ---------- Apply to your DataFrame ----------
df['clean_text'] = df['text'].apply(preprocess)

In [7]:
df.to_csv("stm_cleaned_text.csv", index=False, encoding='utf-8')

In [8]:
df

,file_name,year,text,clean_text
0,AJJacobs_2007P.stm,2007,i've also had some meals that(2) make me want(...,also meal make want dry heave choosing part bi...
1,AaronHuey_2010X.stm,2010,we have indeed {NOISE} taken <sil> the best pa...,indeed taken best part meat let today set phot...
2,AdamGrosser_2007.stm,2007,{NOISE} and then as(2) the(2) ammonia re(2) ev...,ammonia evaporates combine water back erstwhil...
3,AdamSadowsky_2010X.stm,2010,<sil> we came up with(2) {UH} a list {COUGH} o...,came list wanted band integration machine acti...
4,AdamSavage_2008P.stm,2008,and(2) i thought to myself <sil> wouldn't it b...,thought would great dodo skeleton want point p...
...,...,...,...,...
769,YvesBehar_2009.stm,2009,{NOISE} the(2) beginning of any <sil> collabor...,beginning collaboration start conversation wou...
770,ZainabSalbi_2010G.stm,2010,<sil> {SMACK} and {COUGH} killed {COUGH} him <...,killed father kill mother sister soul lie gras...
771,ZeFrank_2004.stm,2004,and {UH} so the idea is {NOISE} that {UH} you ...,idea really know take picture favorite little ...
772,ZeFrank_2010G.stm,2010,{SMACK} this was(2) a mother 's(3) day <sil> (...,mother day thank favorite photo could find pic...


In [9]:
# # NMF clustering, Keywords extract
# n_topics = 10
# nmf_model = NMF(n_components=n_topics, random_state=42)
# W = nmf_model.fit_transform(tfidf)  # text x topic
# H = nmf_model.components_            # topic x word

# # Topic
# df['topic'] = W.argmax(axis=1)

# # Top 10 keywords for each topic
# feature_names = vectorizer.get_feature_names_out()
# for topic_idx, topic in enumerate(H):
#     top_words = [feature_names[i] for i in topic.argsort()[:-11:-1]]
#     print(f"🟢 Topic {topic_idx}: {' | '.join(top_words)}")

In [10]:
# # Use KeyBERT to extract keywords from each document to enhance the understanding of the semantics of each topic
# # Compensate for the limitation of NMF based on word frequency, provide more semantically relevant keywords, and further enhance the understanding of the topic

# kw_model = KeyBERT()
# df['keywords'] = df['clean_text'].apply(
#     lambda x: [kw[0] for kw in kw_model.extract_keywords(x, top_n=5)]
# )

# print(df[['file_name', 'year', 'topic', 'keywords']].head())

In [11]:
# Topic Classification and Keyword Extraction
# # NMF clustering, Keywords extract

import pandas as pd
from sklearn.decomposition import NMF
from keybert import KeyBERT

# TF-IDF vectorization
vectorizer = TfidfVectorizer(
    max_df=0.95, 
    min_df=5, 
    stop_words='english'
)
tfidf = vectorizer.fit_transform(df['clean_text'])

# 1. NMF Topic Modeling
n_topics = 10
nmf_model = NMF(n_components=n_topics, random_state=42)
W = nmf_model.fit_transform(tfidf)  # Document-topic matrix
H = nmf_model.components_           # Topic-word matrix

# Assign the dominant topic to each document
df['topic'] = W.argmax(axis=1)

# Extract top 10 keywords for each topic from NMF components
feature_names = vectorizer.get_feature_names_out()
topic_keywords = []
for topic_idx, topic in enumerate(H):
    top_words = [feature_names[i] for i in topic.argsort()[:-11:-1]]
    topic_keywords.append(top_words)
    print(f"🟢 Topic {topic_idx}: {' | '.join(top_words)}")

🟢 Topic 0: know | thing | people | really | think | kind | going | little | way | say
🟢 Topic 1: cancer | cell | disease | patient | dna | gene | blood | virus | body | genome
🟢 Topic 2: people | africa | country | world | dollar | year | state | percent | market | china
🟢 Topic 3: ocean | sea | water | fish | animal | coral | shark | planet | year | ice
🟢 Topic 4: brain | neuron | human | data | pain | signal | visual | information | child | area
🟢 Topic 5: universe | galaxy | earth | planet | star | space | particle | billion | sun | year
🟢 Topic 6: building | city | design | space | architecture | kind | project | new | idea | work
🟢 Topic 7: school | child | kid | teacher | education | year | english | student | said | learning
🟢 Topic 8: robot | machine | leg | animal | facial | robotics | tail | gecko | foot | expression
🟢 Topic 9: woman | said | story | god | know | men | love | say | mother | man


/Users/jiaxin/opt/anaconda3/envs/5425/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1742: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [12]:
# # Use KeyBERT to extract keywords from each document to enhance the understanding of the semantics of each topic
# # Compensate for the limitation of NMF based on word frequency, provide more semantically relevant keywords, and further enhance the understanding of the topic

# 2. KeyBERT keyword extraction model
kw_model = KeyBERT()

# 3. Extract keywords per document using KeyBERT, excluding keywords that appear in the NMF topic keywords to avoid duplication
def extract_filtered_keywords(text, topic_idx, top_n=5):
    nmf_keys = set(topic_keywords[topic_idx])  # NMF keywords of the document's assigned topic
    kw_keys = kw_model.extract_keywords(text, top_n=top_n*2)  # Extract more candidates to allow filtering
    filtered = []
    for kw, _ in kw_keys:
        if kw not in nmf_keys and kw not in filtered:
            filtered.append(kw)
        if len(filtered) == top_n:
            break
    return filtered

# Apply the filtered keyword extraction to each document
df['keywords'] = df.apply(lambda row: extract_filtered_keywords(row['clean_text'], row['topic']), axis=1)

print(df[['file_name', 'year', 'topic', 'keywords']].head())

                file_name  year  topic  \
0      AJJacobs_2007P.stm  2007      9   
1     AaronHuey_2010X.stm  2010      9   
2    AdamGrosser_2007.stm  2007      0   
3  AdamSadowsky_2010X.stm  2010      8   
4    AdamSavage_2008P.stm  2008      0   

                                            keywords  
0  [bible, leviticus, biblical, sabbath, biblically]  
1         [lakota, sioux, tribe, indigenous, indian]  
2  [ammonia, evaporates, vapor, cooling, refriger...  
3               [music, rhythm, piano, track, audio]  
4               [skeleton, dodo, skull, bone, spend]  


In [13]:
topic_labels = {
    0: "General Ideas & Personal Reflection",            # know, thing, people, think, way
    1: "Medical Science & Disease",                      # cancer, cell, patient, genome
    2: "Global Development & Economics",                 # africa, country, dollar, market, percent
    3: "Marine Life & Environmental Protection",         # ocean, sea, coral, shark, ice
    4: "Neuroscience & Human Cognition",                 # brain, neuron, data, signal, visual
    5: "Astronomy & Space Exploration",                  # universe, galaxy, star, particle, sun
    6: "Architecture & Urban Design",                    # building, design, architecture, city, project
    7: "Education & Child Development",                  # school, child, teacher, learning, student
    8: "Robotics & Biomimicry",                          # robot, machine, leg, gecko, tail
    9: "Gender, Identity & Personal Narratives"          # woman, men, love, mother, god
}

df['topic_name'] = df['topic'].map(topic_labels)

In [14]:
df.to_csv("stm_topic.csv", index=False, encoding='utf-8')

In [15]:
df

,file_name,year,text,clean_text,topic,keywords,topic_name
0,AJJacobs_2007P.stm,2007,i've also had some meals that(2) make me want(...,also meal make want dry heave choosing part bi...,9,"[bible, leviticus, biblical, sabbath, biblically]","Gender, Identity & Personal Narratives"
1,AaronHuey_2010X.stm,2010,we have indeed {NOISE} taken <sil> the best pa...,indeed taken best part meat let today set phot...,9,"[lakota, sioux, tribe, indigenous, indian]","Gender, Identity & Personal Narratives"
2,AdamGrosser_2007.stm,2007,{NOISE} and then as(2) the(2) ammonia re(2) ev...,ammonia evaporates combine water back erstwhil...,0,"[ammonia, evaporates, vapor, cooling, refriger...",General Ideas & Personal Reflection
3,AdamSadowsky_2010X.stm,2010,<sil> we came up with(2) {UH} a list {COUGH} o...,came list wanted band integration machine acti...,8,"[music, rhythm, piano, track, audio]",Robotics & Biomimicry
4,AdamSavage_2008P.stm,2008,and(2) i thought to myself <sil> wouldn't it b...,thought would great dodo skeleton want point p...,0,"[skeleton, dodo, skull, bone, spend]",General Ideas & Personal Reflection
...,...,...,...,...,...,...,...
769,YvesBehar_2009.stm,2009,{NOISE} the(2) beginning of any <sil> collabor...,beginning collaboration start conversation wou...,6,"[invention, tesla, collaboration, conversation...",Architecture & Urban Design
770,ZainabSalbi_2010G.stm,2010,<sil> {SMACK} and {COUGH} killed {COUGH} him <...,killed father kill mother sister soul lie gras...,9,"[sarajevo, survivor, genocide, survived, samia]","Gender, Identity & Personal Narratives"
771,ZeFrank_2004.stm,2004,and {UH} so the idea is {NOISE} that {UH} you ...,idea really know take picture favorite little ...,0,"[minister, puppet, rebel, puppeteer, guru]",General Ideas & Personal Reflection
772,ZeFrank_2010G.stm,2010,{SMACK} this was(2) a mother 's(3) day <sil> (...,mother day thank favorite photo could find pic...,0,"[stress, stressed, trend, affect, disturbed]",General Ideas & Personal Reflection


In [16]:
# Visualization Version 1

import plotly.express as px
import pandas as pd

# Ensure 'year' is of integer type
df['year'] = df['year'].astype(int)

# Aggregate the number of talks per year per topic
topic_trend = df.groupby(['year', 'topic_name']).size().reset_index(name='count')

# Plot: Interactive line chart
fig = px.line(
    topic_trend,
    x="year",
    y="count",
    color="topic_name",
    markers=True,
    title="📊 TED Talk Topics Over Time",
    labels={
        "year": "Year",
        "count": "Number of Talks",
        "topic_name": "Topic"
    },
    width=1200,  # Increase chart width
    height=600
)

# Customize axis and layout
fig.update_layout(
    xaxis=dict(
        dtick=1,           # One tick per year
        tickangle=45,      # Rotate year labels for clarity
        tickfont=dict(size=10)  # Optional: adjust font size
    ),
    legend_title="Topic Name",
    hovermode="x unified"
)

# Display the chart
fig.show()

In [17]:
# Visualization Version 2

import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import base64
from io import BytesIO

# Assuming your DataFrame is already loaded
# df = pd.read_csv('path_to_data.csv')

# Prepare dropdown options
all_topics = sorted(df['topic_name'].unique())
min_year, max_year = df['year'].min(), df['year'].max()

# ------------------------
# Create Dash app
# ------------------------
app = Dash(__name__)
app.title = "TED Topic Explorer"

app.layout = html.Div([
    html.H1("🎤 TED Talk Topic Explorer", style={'textAlign': 'center', 'color': '#ffffff', 'marginTop': '20px'}),

    html.Div([
        html.Label("📅 Year Range", style={'color': '#ffffff'}),
        dcc.RangeSlider(
            min=min_year,
            max=max_year,
            value=[min_year, max_year],
            marks={i: str(i) for i in range(min_year, max_year+1, 2)},
            id='year-slider'
        )
    ], style={'width': '80%', 'display': 'inline-block', 'padding': '20px'}),

    html.Div([
        dcc.Graph(id='trend-graph')
    ]),

    html.Div([
        html.Div([
            html.Label("🧠 Select Topic", style={'color': '#ffffff'}),
            dcc.Dropdown(
                options=[{'label': t, 'value': t} for t in all_topics],
                value=all_topics[0],
                id='topic-dropdown'
            )
        ], style={'marginBottom': '20px'}),

        html.H3("🔠 Keyword Word Cloud", style={'textAlign': 'center', 'color': '#ffffff'}),

        html.Img(
            id='wordcloud',
            style={
                'height': '600px',
                'width': '800px',
                'display': 'block',
                'margin': '0 auto'
            }
        )

    ], style={
        'marginTop': '30px',
        'backgroundColor': '#2a2a40',
        'padding': '20px',
        'borderRadius': '10px',
        'boxShadow': '0px 0px 10px rgba(255, 255, 255, 0.1)'
    }),

    html.Div(id='sample-texts', style={'padding': '30px', 'color': '#ffffff'})
], style={
    'backgroundColor': '#1e1e2f',
    'minHeight': '100vh',
    'padding': '20px',
    'fontFamily': 'Arial, sans-serif'
})

# ------------------------
# Callback: Update trend chart
# ------------------------
@app.callback(
    Output('trend-graph', 'figure'),
    Input('year-slider', 'value'),
)
def update_trend(year_range):
    filtered = df[(df['year'] >= year_range[0]) & (df['year'] <= year_range[1])]
    trend = filtered.groupby(['year', 'topic_name']).size().reset_index(name='count')
    fig = px.line(
        trend,
        x="year", y="count", color="topic_name", markers=True,
        title="📈 Topic Trend Over Time"
    )
    fig.update_layout(xaxis=dict(dtick=1),
    xaxis_tickangle=-45)
    return fig

# ------------------------
# Callback: Generate word cloud
# ------------------------
@app.callback(
    Output('wordcloud', 'src'),
    Input('topic-dropdown', 'value')
)
def update_wordcloud(topic_name):
    topic_keywords = df[df['topic_name'] == topic_name]['keywords'].explode()
    freq = topic_keywords.value_counts().to_dict()

    wordcloud = WordCloud(width=800, height=600, background_color='white', max_words=200).generate_from_frequencies(freq)

    buf = BytesIO()
    plt.figure(figsize=(10, 8), dpi=300)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.tight_layout()
    plt.savefig(buf, format='png')
    plt.close()

    encoded = base64.b64encode(buf.getvalue()).decode()
    return f"data:image/png;base64,{encoded}"

# ------------------------
# Callback: Display sample texts
# ------------------------
@app.callback(
    Output('sample-texts', 'children'),
    Input('topic-dropdown', 'value')
)
def show_samples(topic_name):
    samples = df[df['topic_name'] == topic_name].sample(3, random_state=1)
    children = []
    for _, row in samples.iterrows():
        snippet = row['text'][:300].replace('\n', ' ') + "..."
        clean_snippet = row['clean_text'][:300].replace('\n', ' ') + "..."
        children.append(html.Div([
            html.H4(f"🎙️ {row['file_name']} ({row['year']})"),
            html.P(snippet),
            html.P(f"🔑 Keywords: {', '.join(row['keywords'])}", style={'color': 'gray'}),
            html.P("🧹 Cleaned Text:", style={'fontWeight': 'bold', 'marginTop': '10px', 'color': '#FFD700'}),
            html.P(clean_snippet, style={'fontStyle': 'italic', 'color': '#DDDDDD'})
        ], style={'marginBottom': '30px'}))
    return children

# ------------------------
# Launch the server
# ------------------------
if __name__ == '__main__':
    app.run(debug=True)
    
